# 의사결정 나무모델    

- 지도학습 알고리즘 (분류, 회귀)
- 직관적인 알고리즘 (이해 쉬움)
- 과대적합되기 쉬운 알고리즘 (트리 깊이 제한 필요)


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
# 데이터 생성
from sklearn.datasets import load_breast_cancer

def make_dataset():
    iris = load_breast_cancer()
    df = pd.DataFrame(iris.data, columns=iris.feature_names)
    df['target'] = iris.target
    X_train, X_test, y_train, y_test = train_test_split(
        df.drop('target', axis=1), df['target'], test_size=0.5, random_state=1004)
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = make_dataset()
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((284, 30), (285, 30), (284,), (285,))

In [3]:
# 타겟 확인
y_train.value_counts()

1    190
0     94
Name: target, dtype: int64

# 2. 의사결정나무
- 지도학습(분류)에서 가장 유용하게 사용되고 있는 기법 중 하나입니다.
- 트리의 루트(root)에서 시작해서 정보이득이 최대가 되는 특성으로 데이터를 나눕니다.
- 정보이득(information gain)이 최대가 되는 특성을 나누는 기준(불순도를 측정하는 기준)은 '지니'와 '엔트로피'가 사용됩니다.
- 데이터가 한 종류만 있다면 엔트로피/지니 불순도는 0에 가깝고, 서로 다른 데이터의 비율이 비슷하면 1에 가깝습니다.
- 정보이득(information gain)이 최대라는 것은 불순도를 최소화 하는 방향입니다. (1-불순도)


In [4]:
# 의사결정나무
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(random_state=0)
model.fit(X_train, y_train)
pred = model.predict(X_test)
accuracy_score(y_test, pred)

0.9263157894736842

# 3. 의사결정나무 하이퍼파라미터
- criterion (기본값 gini) : 불순도 지표 (또는 엔트로피 불순도 entropy)
- max_depth (기본값 None) : 최대 한도 깊이
- min_samples_split (기본값 2) : 자식 노드를 갖기 위한 최소한의 데이터 수
- min_samples_leaf (기본값 1) : 리프 노드가 되기 위한 최소 샘플 수

In [15]:
# 의사결정나무 하이퍼파라미터
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(
    criterion = 'entropy',
    max_depth = 4,
    min_samples_split = 2,
    min_samples_leaf = 2,
    random_state = 0)
model.fit(X_train, y_train)
pred = model.predict(X_test)
accuracy_score(y_test, pred)

0.9403508771929825

# 랜덤포레스트

- 여러개의 의사결정 트리로 구성
- 앙상블 방법 중 배깅(bagging) 방식
- 부트스트랩 샘플링 (데이터셋 중복 허용)
- 최종 다수결 투표
- 과대적합 가능성 낮음

## 앙상블 방법

-배깅: 같은 알고리즘으로 여러 모델을 만들어 분류함(랜덤포레스트)

-부스팅: 학습과 예측을 하면서 가중치 반영 (xgboost)

In [17]:
# 랜덤포레스트
from sklearn.ensemble import RandomForestClassifier
# 모델 선택
model = RandomForestClassifier(random_state=0)
model.fit(X_train, y_train)
pred = model.predict(X_test)
accuracy_score(y_test, pred)

0.9438596491228071

# 랜덤포레스트의 하이퍼파라미터
- n_estimators (기본값 100) : 트리의 수
- criterion (기본값 gini) : 불순도 지표
- max_depth (기본값 None) : 최대 한도 깊이
- min_samples_split (기본값 2) : 자식 노드를 갖기 위한 최소한의 데이터 수
- min_samples_leaf (기본값 1) : 리프 노드가 되기 위한 최소 샘플 수


In [20]:
# 랜덤포레스트
from sklearn.ensemble import RandomForestClassifier
# 모델 선택
model = RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0)
model.fit(X_train, y_train)
pred = model.predict(X_test)
accuracy_score(y_test, pred)

0.9473684210526315

# XGBoost
- 트리 앙상블 중 성능이 좋은 알고리즘
- eXtreme Gradient Boosting를 줄여서 XGBoost라고 한다.
- 약한 학습기가 계속해서 업데이트를 하며 좋은 모델을 만들어 간다.
- 부스팅(앙상블) 기반의 알고리즘
- 캐글(글로벌 AI 경진대회)에서 뛰어난 성능을 보이면서 인기가 높아짐

In [24]:
from xgboost import XGBClassifier
model = XGBClassifier(random_state=0, use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)
pred = model.predict(X_test)
accuracy_score(y_test, pred)

# use_label_encoder=False, eval_metric='logloss'는 오류 제거용

0.9508771929824561

# xgboost 하이퍼 파라미터

- booster (기본값 gbtree) : 부스팅 알고리즘 (또는 dart, gblinear)
- objective (기본값 binary:logistic) : 이진분류 (다중분류: multi:softmax)
- max_depth (기본값 6) : 최대 한도 깊이
- learning_rate (기본값 0.1) : 학습률
- n_estimators (기본값 100) : 트리의 수
- subsample (기본값 1) : 훈련 샘플 개수의 비율
- colsample_bytree (기본값 1) : 특성 개수의 비율
- n_jobs (기본값 1) : 사용 코어 수 (-1: 모든 코어를 다 사용)

# *learning_rate와 n_estimators 값은 반대방향으로 같이 움직여줘야함

In [29]:
# xgboost 하이퍼 파라미터
model = XGBClassifier(random_state=0, use_label_encoder=False, eval_metric='logloss',
                     booster = 'gbtree',
                     objective = 'binary:logistic',
                     max_depth = 5,
                     learning_rate = 0.1,
                     n_estimators = 500,
                     subsample = 1,
                     colsample_bytree = 1,
                     n_jobs = -1)
model.fit(X_train, y_train)
pred = model.predict(X_test)
accuracy_score(y_test, pred)


# - booster (기본값 gbtree) : 부스팅 알고리즘 (또는 dart, gblinear)
# - objective (기본값 binary:logistic) : 이진분류 (다중분류: multi:softmax)
# - max_depth (기본값 6) : 최대 한도 깊이
# - learning_rate (기본값 0.1) : 학습률
# - n_estimators (기본값 100) : 트리의 수
# - subsample (기본값 1) : 훈련 샘플 개수의 비율
# - colsample_bytree (기본값 1) : 특성 개수의 비율
# - n_jobs (기본값 1) : 사용 코어 수 (-1: 모든 코어를 다 사용)

# 0.9614035087719298 (n_estimators = 100)
# 0.9649122807017544 (n_estimators = 500)

0.9649122807017544

In [31]:
# 조기종료
model = XGBClassifier(random_state=0, use_label_encoder=False, eval_metric='logloss',
                     learning_rate = 0.1,
                     n_estimators = 500)

# 검증할 데이터셋 추가
eval_set = [(X_test, y_test)]
model.fit(X_train, y_train, eval_set=eval_set, early_stopping_rounds=10) # 학습 검증할 데이터셋 추가, 성능 개선 없을 경우 조기 종료
pred = model.predict(X_test)
accuracy_score(y_test, pred)

[0]	validation_0-logloss:0.61675
[1]	validation_0-logloss:0.55412
[2]	validation_0-logloss:0.50327
[3]	validation_0-logloss:0.46000
[4]	validation_0-logloss:0.42249
[5]	validation_0-logloss:0.39066
[6]	validation_0-logloss:0.36314
[7]	validation_0-logloss:0.33875
[8]	validation_0-logloss:0.31792
[9]	validation_0-logloss:0.29937
[10]	validation_0-logloss:0.28334
[11]	validation_0-logloss:0.26923
[12]	validation_0-logloss:0.25589
[13]	validation_0-logloss:0.24533
[14]	validation_0-logloss:0.23600
[15]	validation_0-logloss:0.22732
[16]	validation_0-logloss:0.22058
[17]	validation_0-logloss:0.21358
[18]	validation_0-logloss:0.20829
[19]	validation_0-logloss:0.20228
[20]	validation_0-logloss:0.19858
[21]	validation_0-logloss:0.19410
[22]	validation_0-logloss:0.19093
[23]	validation_0-logloss:0.18808
[24]	validation_0-logloss:0.18447
[25]	validation_0-logloss:0.18285
[26]	validation_0-logloss:0.18104
[27]	validation_0-logloss:0.17707
[28]	validation_0-logloss:0.17486
[29]	validation_0-loglos

0.9543859649122807

# 교차검증
- 일반적으로 모델을 학습시킬 때 데이터를 train set과 test set으로 나누어 train set을 가지고 학습을 수행합니다.
- 교차검증이란 여기서 train set을 다시 train set과 validation set으로 나누어 학습 중 검증과 수정을 수행하는 것을 의미합니다.


In [33]:
# 데이터셋 로드
def make_dataset2():
    iris = load_breast_cancer()
    df = pd.DataFrame(iris.data, columns=iris.feature_names)
    df['target'] = iris.target
    return df.drop('target', axis=1), df['target']
X, y = make_dataset2()

In [35]:
from sklearn.model_selection import KFold
model = DecisionTreeClassifier(random_state=0)

kfold = KFold(n_splits=5)
for train_idx, test_idx in kfold.split(X):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    print(accuracy_score(y_test, pred))

0.8771929824561403
0.9122807017543859
0.9473684210526315
0.9385964912280702
0.8407079646017699


# Stratified Kfold
- 불균형한 타겟 비율을 가진 데이터가 한쪽으로 치우치는 것을 방지

In [36]:
# Stratified Kfold
from sklearn.model_selection import StratifiedKFold
model = DecisionTreeClassifier(random_state=0)

kfold = StratifiedKFold(n_splits=5)
for train_idx, test_idx in kfold.split(X,y): # y값까지 추가해줘야함!
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    print(accuracy_score(y_test, pred))

0.9035087719298246
0.9210526315789473
0.9122807017543859
0.9473684210526315
0.9026548672566371


# 사이킷런 교차검증
- 사이킷런 내부 API를 통해 FIT(학습) - PREDICT(예측) - EVALUATION(평가)

In [37]:
# 교차검증
# API 불러오기
from sklearn.model_selection import cross_val_score
cross_val_score(model, X, y, cv=3) # cv뒤 숫자를 바꿔주면 몇개로 데이터를 쪼갤 것인지 다양한 검증 가능

array([0.88947368, 0.94210526, 0.86243386])

In [38]:
# cross_val_score의 평균 점수 확인
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model, X, y, cv=3)  # 해당 함수를 scores로 저장
scores.mean()

0.8980042699340944

In [39]:
# 교차검증 StratifiedKFold
kfold = StratifiedKFold(n_splits=5)
scores = cross_val_score(model, X, y, cv=kfold)
scores

array([0.90350877, 0.92105263, 0.9122807 , 0.94736842, 0.90265487])

In [40]:
# 평균
scores.mean()

0.9173730787144851

# 평가(분류)
- 정확도 accuracy: 실제 값과 예측값이 일치하는 비율
- 정밀도 precision: 양성이라고 예측한 값 중 실제 양성인 값의 비율 (암이라고 예측 한 값 중 실제 암)
- 재현율 recall: 실제 양성 값 중 양성으로 예측한 값의 비율 (암을 암이라고 판단)
- F1: 정밀도와 재현율의 조화평균
- ROC-AUC
- ROC: 참 양성 비율(True Positive Rate)에 대한 거짓 양성 비율(False Positive Rate) 곡선
- AUC: ROC곡선 면적 아래 (완벽하게 분류되면 AUC가 1임)

In [41]:
# 정확도
from sklearn.metrics import accuracy_score
accuracy_score(y_test, pred)

0.9026548672566371

In [42]:
# 정밀도
from sklearn.metrics import precision_score
precision_score(y_test, pred)

0.9545454545454546

In [43]:
# 재현율
from sklearn.metrics import recall_score
recall_score(y_test, pred)

0.8873239436619719

In [44]:
# f1
from sklearn.metrics import f1_score
f1_score(y_test, pred)

0.9197080291970803

In [45]:
# roc_auc
from sklearn.metrics import roc_auc_score
model = XGBClassifier(random_state=0, use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)
pred = model.predict_proba(X_test) # predict가 아니라 predict_proba(0,1이 아니라 확률로 받음)
roc_auc_score(y_test, pred[:,1])

0.999664654594232

In [46]:
pip install graphviz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 KB 2.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
